<a href="https://colab.research.google.com/github/tyoc213/fastai_xla_extensions/blob/explorations1/explore_nbs/RNN_test_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
VERSION = "20200707" #"nightly"  #"20200515" @param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py > /dev/null
!python pytorch-xla-env-setup.py
import torch_xla.core.xla_model as xm

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5115  100  5115    0     0  88189      0 --:--:-- --:--:-- --:--:-- 88189
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
Uninstalling torch-1.6.0:
  Successfully uninstalled torch-1.6.0
Uninstalling torchvision-0.7.0:
  Successfully uninstalled torchvision-0.7.0
Copying gs://tpu-pytorch/wheels/torch-nightly+20200515-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 91.0 MiB/ 91.0 MiB]                                                
Operation completed over 1 objects/91.0 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200515-cp36-cp36m-linux_x86_64.whl...
/ [1 files][119.5 MiB/119.5 MiB]                                                
Operation completed over 1 objects/119.5 MiB.                                    
Copying gs://tpu-pytorch/wh

In [2]:
#!pip install https://github.com/butchland/fastai_xla_extensions/archive/master.zip
#import fastai_xla_extensions.core
#XLA_AVAILABLE

In [3]:
#!pip install https://github.com/fastai/fastcore/archive/master.zip
#!pip install https://github.com/fastai/fastai2/archive/master.zip

In [4]:
from fastai2.text.all import *

In [11]:

class XLAOptimProxy:
    def __init__(self,opt:Optimizer):
        self.opt = opt

    def xla_step(self):
        xm.optimizer_step(self.opt,barrier=True) # sync on gradient update

    def __getattr__(self,name):
        if name == 'step': # override proxying for step method
                return getattr(self,'xla_step')
        # proxy everything else
        return getattr(self.opt,name)
@patch_to(Learner)
def create_opt(self):
        ooo = self.opt_func(self.splitter(self.model), lr=self.lr)
        prox = XLAOptimProxy(ooo)
        self.opt = prox
        if not self.wd_bn_bias:
            for p in self._bn_bias_state(True ): p['do_wd'] = False
        if self.train_bn:
            for p in self._bn_bias_state(False): p['force_train'] = True

In [12]:
class LMModel1(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        
    def forward(self, x):
        h = F.relu(self.h_h(self.i_h(x[:,0])))
        h = h + self.i_h(x[:,1])
        h = F.relu(self.h_h(h))
        h = h + self.i_h(x[:,2])
        h = F.relu(self.h_h(h))
        return self.h_o(h)

In [13]:
path = untar_data(URLs.HUMAN_NUMBERS)

Path.BASE_PATH = path

lines = L()
with open(path/'train.txt') as f: lines += L(*f.readlines())
with open(path/'valid.txt') as f: lines += L(*f.readlines())


text = ' . '.join([l.strip() for l in lines])
tokens = text.split(' ')
tokens = text.split(' ')
vocab = L(*tokens).unique()
word2idx = {w:i for i,w in enumerate(vocab)}
nums = L(word2idx[i] for i in tokens)

In [14]:
seqs = L((tensor(nums[i:i+3]), nums[i+3]) for i in range(0,len(nums)-4,3))
bs = 64
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(seqs[:cut], seqs[cut:], bs=64, shuffle=False, device=xm.xla_device())

In [15]:
print(dls.device)

xla:1


In [16]:
%%time

learn = Learner(dls, LMModel1(len(vocab), 64), loss_func=F.cross_entropy, 
                metrics=accuracy)
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.817019,1.808402,0.481578,00:11


CPU times: user 4.37 s, sys: 780 ms, total: 5.15 s
Wall time: 11.6 s


# Our first recurrent NN

In [17]:
class LMModel2(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        
    def forward(self, x):
        h = 0
        for i in range(3):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
        return self.h_o(h)

In [18]:
dls.device

device(type='xla', index=1)

In [19]:
%%time

learn = Learner(dls, LMModel2(len(vocab), 64), loss_func=F.cross_entropy, 
                metrics=accuracy)
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.765053,1.732569,0.481816,00:11


CPU times: user 4.64 s, sys: 866 ms, total: 5.51 s
Wall time: 11.9 s


# Maintain state

In [20]:
class LMModel3(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
        
    def forward(self, x):
        for i in range(3):
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
        out = self.h_o(self.h)
        self.h = self.h.detach()
        return out
    
    def reset(self): self.h = 0

In [21]:
%%time

learn = Learner(dls, LMModel3(len(vocab), 64), loss_func=F.cross_entropy, 
                metrics=accuracy)
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.821301,1.810582,0.489663,01:51


AssertionError: ignored

# use ModelResetter Callback

In [24]:
%%time
learn = Learner(dls, LMModel3(len(vocab), 64), loss_func=F.cross_entropy,
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(10, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.895206,1.912353,0.459375,01:54


AssertionError: ignored

#Model4

In [25]:
class LMModel4(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
        
    def forward(self, x):
        outs = []
        for i in range(sl):
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
            outs.append(self.h_o(self.h))
        self.h = self.h.detach()
        return torch.stack(outs, dim=1)
    
    def reset(self): self.h = 0

def loss_func(inp, targ):
    return F.cross_entropy(inp.view(-1, len(vocab)), targ.view(-1))

In [27]:
%%time
learn = Learner(dls, LMModel4(len(vocab), 64), loss_func=loss_func,
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(1, 3e-3)

NameError: ignored

# Model5

In [28]:
class LMModel5(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.RNN(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = torch.zeros(n_layers, bs, n_hidden)
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(res)
    
    def reset(self): self.h.zero_()

In [ ]:
%%time
learn = Learner(dls, LMModel5(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(1, 3e-3)

epoch,train_loss,valid_loss,accuracy,time


# The above crashed my session with the following log:


```
Jul 31, 2020, 11:48:13 PM	WARNING	WARNING:root:kernel 3f9019c4-da8e-45f3-8546-44d6f18cb4f6 restarted
Jul 31, 2020, 11:48:13 PM	INFO	KernelRestarter: restarting kernel (2/5), keep random ports
Jul 31, 2020, 11:48:11 PM	WARNING	zmq.error.ZMQError: Address already in use
Jul 31, 2020, 11:48:11 PM	WARNING	File "zmq/backend/cython/checkrc.pxd", line 26, in zmq.backend.cython.checkrc._check_rc
Jul 31, 2020, 11:48:11 PM	WARNING	File "zmq/backend/cython/socket.pyx", line 550, in zmq.backend.cython.socket.Socket.bind
Jul 31, 2020, 11:48:11 PM	WARNING	s.bind("tcp://%s:%i" % (self.ip, port))
Jul 31, 2020, 11:48:11 PM	WARNING	File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelapp.py", line 181, in _bind_socket
Jul 31, 2020, 11:48:11 PM	WARNING	self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
Jul 31, 2020, 11:48:11 PM	WARNING	File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelapp.py", line 239, in init_sockets
Jul 31, 2020, 11:48:11 PM	WARNING	self.init_sockets()
Jul 31, 2020, 11:48:11 PM	WARNING	File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelapp.py", line 469, in initialize
Jul 31, 2020, 11:48:11 PM	WARNING	return method(app, *args, **kwargs)
Jul 31, 2020, 11:48:11 PM	WARNING	File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 87, in catch_config_error
Jul 31, 2020, 11:48:11 PM	WARNING	File "<decorator-gen-121>", line 2, in initialize
Jul 31, 2020, 11:48:11 PM	WARNING	app.initialize(argv)
Jul 31, 2020, 11:48:11 PM	WARNING	File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 663, in launch_instance
Jul 31, 2020, 11:48:11 PM	WARNING	app.launch_new_instance()
Jul 31, 2020, 11:48:11 PM	WARNING	File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
Jul 31, 2020, 11:48:11 PM	WARNING	exec(code, run_globals)
Jul 31, 2020, 11:48:11 PM	WARNING	File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
Jul 31, 2020, 11:48:11 PM	WARNING	"__main__", mod_spec)
Jul 31, 2020, 11:48:11 PM	WARNING	File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
Jul 31, 2020, 11:48:11 PM	WARNING	Traceback (most recent call last):
Jul 31, 2020, 11:48:10 PM	WARNING	WARNING:root:kernel 3f9019c4-da8e-45f3-8546-44d6f18cb4f6 restarted
Jul 31, 2020, 11:48:10 PM	INFO	KernelRestarter: restarting kernel (1/5), keep random ports
Jul 31, 2020, 11:48:08 PM	WARNING	cpu:0
Jul 31, 2020, 11:48:08 PM	WARNING	*** End stack trace ***
Jul 31, 2020, 11:48:08 PM	WARNING	torch_xla::bridge::SetCurrentDevice(c10::Device const&)
Jul 31, 2020, 11:48:08 PM	WARNING	torch_xla::bridge::AtenDeviceToXlaDevice(c10::Device const&)
Jul 31, 2020, 11:48:08 PM	WARNING	tensorflow::CurrentStackTrace[abi:cxx11]()
Jul 31, 2020, 11:48:08 PM	WARNING	*** Begin stack trace ***
Jul 31, 2020, 11:48:08 PM	WARNING	what(): torch_xla/csrc/aten_xla_bridge.cpp:182 : Check failed: device.type() == at::kXLA (cpu vs. xla)
Jul 31, 2020, 11:48:08 PM	WARNING	terminate called after throwing an instance of 'std::runtime_error'
Jul 31, 2020, 11:48:08 PM	WARNING	cpu:0
Jul 31, 2020, 11:48:08 PM	WARNING	*** End stack trace ***
Jul 31, 2020, 11:48:08 PM	WARNING	torch_xla::bridge::SetCurrentDevice(c10::Device const&)
Jul 31, 2020, 11:48:08 PM	WARNING	torch_xla::bridge::AtenDeviceToXlaDevice(c10::Device const&)
Jul 31, 2020, 11:48:08 PM	WARNING	tensorflow::CurrentStackTrace[abi:cxx11]()
Jul 31, 2020, 11:48:08 PM	WARNING	*** Begin stack trace ***
Jul 31, 2020, 11:48:08 PM	WARNING	2020-08-01 04:48:08.352531: E tensorflow/compiler/xla/xla_client/tf_logging.cc:11] Check failed: device.type() == at::kXLA (cpu vs. xla)
Jul 31, 2020, 11:45:35 PM	WARNING	(64, 64) and (64, 64)
Jul 31, 2020, 11:45:35 PM	WARNING	*** End stack trace ***
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	PyObject_Call
Jul 31, 2020, 11:45:35 PM	WARNING	_PyFunction_FastCallDict
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	PyObject_Call
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	PyObject_Call
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	PyObject_Call
Jul 31, 2020, 11:45:35 PM	WARNING	_PyFunction_FastCallDict
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	PyObject_Call
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	PyObject_Call
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	PyObject_Call
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	PyObject_Call
Jul 31, 2020, 11:45:35 PM	WARNING	_PyFunction_FastCallDict
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	PyObject_Call
Jul 31, 2020, 11:45:35 PM	WARNING	_PyFunction_FastCallDict
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	PyObject_Call
Jul 31, 2020, 11:45:35 PM	WARNING	_PyFunction_FastCallDict
Jul 31, 2020, 11:45:35 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:45:35 PM	WARNING	PyNumber_Add
Jul 31, 2020, 11:45:35 PM	WARNING	PyObject_Call
Jul 31, 2020, 11:45:35 PM	WARNING	PyCFunction_Call
Jul 31, 2020, 11:45:35 PM	WARNING	c10::impl::wrap_kernel_functor_unboxed_<c10::impl::detail::WrapFunctionIntoRuntimeFunctor_<at::Tensor (*)(at::Tensor const&, at::Tensor const&, c10::Scalar), at::Tensor, c10::guts::typelist::typelist<at::Tensor const&, at::Tensor const&, c10::Scalar> >, at::Tensor (at::Tensor const&, at::Tensor const&, c10::Scalar)>::call(c10::OperatorKernel*, at::Tensor const&, at::Tensor const&, c10::Scalar)
Jul 31, 2020, 11:45:35 PM	WARNING	torch_xla::AtenXlaType::add(at::Tensor const&, at::Tensor const&, c10::Scalar)
Jul 31, 2020, 11:45:35 PM	WARNING	torch_xla::XLATensor::add(torch_xla::XLATensor const&, torch_xla::XLATensor const&, c10::Scalar, c10::optional<c10::ScalarType>)
Jul 31, 2020, 11:45:35 PM	WARNING	torch_xla::ir::operator+(torch_xla::ir::Value const&, torch_xla::ir::Value const&)
Jul 31, 2020, 11:45:35 PM	WARNING	torch_xla::XlaHelpers::GetPromotedBinaryOpShape(xla::Shape const&, xla::Shape const&)
Jul 31, 2020, 11:45:35 PM	WARNING	torch_xla::XlaHelpers::GetPromotedShape(absl::lts_2020_02_25::Span<long long const>, absl::lts_2020_02_25::Span<long long const>)
Jul 31, 2020, 11:45:35 PM	WARNING	tensorflow::CurrentStackTrace[abi:cxx11]()
Jul 31, 2020, 11:45:35 PM	WARNING	*** Begin stack trace ***
Jul 31, 2020, 11:42:32 PM	WARNING	(64, 64) and (64, 64)
Jul 31, 2020, 11:42:32 PM	WARNING	*** End stack trace ***
Jul 31, 2020, 11:42:32 PM	WARNING	_PyEval_EvalFrameDefault
Jul 31, 2020, 11:42:32 PM	WARNING	PyObject_Call

```